## Configs and Hyperparameters

Support a variety of models, you can find more pretrained model from [Tensorflow detection model zoo: COCO-trained models](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/detection_model_zoo.md#coco-trained-models), as well as their pipline config files in [object_detection/samples/configs/](https://github.com/tensorflow/models/tree/master/research/object_detection/samples/configs).

In [1]:
import tensorflow as tf
print(tf.test.is_built_with_cuda())
print(tf.config.list_physical_devices('GPU')) 

True
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
import sys
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\')
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\research')
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\research\\object_detection')
sys.path.insert(0, 'D:\\Project\\AiTalents\\models\\research\\slim')
print(sys.path)

['D:\\Project\\AiTalents\\models\\research\\slim', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\', 'D:\\Project\\AiTalents', 'D:\\Project\\AiTalents\\models', 'D:\\Project\\AiTalents\\models\\research', 'D:\\Project\\AiTalents\\models\\research\\object_detection', 'D:\\Project\\AiTalents\\models\\research\\slim', 'D:\\Project\\AiTalents', 'C:\\Users\\User\\anaconda3\\envs\\dl\\python38.zip', 'C:\\Users\\User\\anaconda3\\envs\\dl\\DLLs', 'C:\\Users\\User\\anaconda3\\envs\\dl\\lib', 'C:\\Users\\User\\anaconda3\\envs\\dl', '', 'C:\\Users\\User\\AppData\\Roaming\\Python\\Python38\\site-packages', 'C:\\Users\\User\\anaconda3\\envs\\dl\\lib\\site-packages', 'C:\\Users\\User\\anaconda3\\envs\\dl\\lib\\site-packages\\win32', 'C:\\Users\\User\\anaconda3\\envs\\dl\\lib\\site-packages\\win32\\lib', 'C:\\Users\\User\\anaconda3\\envs\\dl\\lib\\site-packages\\Pythonwin', 'C:\\Users\\User\\anaconda3\\envs\\d

In [3]:
# Number of training steps - 1000 will train very quickly, but more steps will increase accuracy.
num_steps = 10000  # 200000 to improve

# Number of evaluation steps.
num_eval_steps = 50

MODELS_CONFIG = {
    'ssd_mobilenet_v2': {
        'model_name': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8',
        'pipeline_file': 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.config',
        'batch_size': 12
    }, 
}

# Pick the model you want to use
# Select a model in `MODELS_CONFIG`.
selected_model = 'ssd_mobilenet_v2'

# Name of the object detection model to use.
MODEL = MODELS_CONFIG[selected_model]['model_name']

# Name of the pipline file in tensorflow object detection API.
pipeline_file = MODELS_CONFIG[selected_model]['pipeline_file']

# Training batch size fits in Colabe's Tesla K80 GPU memory for selected model.
batch_size = MODELS_CONFIG[selected_model]['batch_size']

## Install required packages

In [4]:
# %cd /content
# !git clone --quiet https://github.com/tensorflow/models.git
# %cd D:/Project/AiTalents
!pip install tf_slim

# !apt-get install -qq protobuf-compiler python-pil python-lxml python-tk

!pip install -q Cython contextlib2 pillow lxml matplotlib

!pip install -q pycocotools

!pip install lvis

!pip install opencv-python

# %cd models/research/
# %SET PYTHONPATH=%cd%;%cd%\slim
# !protoc object_detection/protos/*.proto --python_out=.

# import os
# os.environ['PYTHONPATH'] += ':models/research/:models/research/slim/'


!python models/research/object_detection/builders/model_builder_test.py

2020-12-20 05:13:53.560455: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll


## Prepare `tfrecord` files

Roboflow automatically creates our TFRecord and label_map files that we need!

**Generating your own TFRecords the only step you need to change for your own custom dataset.**

Because we need one TFRecord file for our training data, and one TFRecord file for our test data, we'll create two separate datasets in Roboflow and generate one set of TFRecords for each.

To create a dataset in Roboflow and generate TFRecords, follow [this step-by-step guide](https://blog.roboflow.ai/getting-started-with-roboflow/).

In [5]:
# NOTE: Update these TFRecord names from "cells" and "cells_label_map" to your files!
test_record_fname = 'D:/Project/AiTalents/test.tfrecord'
train_record_fname = 'D:/Project/AiTalents/train.tfrecord'
label_map_pbtxt_fname = 'D:/Project/AiTalents/class.pbtxt'
checkpoint_type = 'detection'

## Download base model

In [6]:
import os
import shutil
import glob
import urllib.request
import tarfile
MODEL_FILE = MODEL + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
DEST_DIR = 'models/research/pretrained_model/'

if not (os.path.exists(MODEL_FILE)):
    urllib.request.urlretrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)

tar = tarfile.open(MODEL_FILE)
tar.extractall()
tar.close()

os.remove(MODEL_FILE)
if (os.path.exists(DEST_DIR)):
    shutil.rmtree(DEST_DIR)
os.rename(MODEL, DEST_DIR)

In [6]:
# fine_tune_checkpoint = os.path.join(DEST_DIR, "model.ckpt")
fine_tune_checkpoint = "D:/Project/AiTalents/models/research/pretrained_model/checkpoint/ckpt-0"
fine_tune_checkpoint

'D:/Project/AiTalents/models/research/pretrained_model/checkpoint/ckpt-0'

## Configuring a Training Pipeline

In [7]:
import os
pipeline_fname = os.path.join('D:/Project/AiTalents/models/research/object_detection/configs/tf2/', pipeline_file)

assert os.path.isfile(pipeline_fname), '`{}` not exist'.format(pipeline_fname)

In [8]:
def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())

In [9]:
import re

num_classes = get_num_classes(label_map_pbtxt_fname)
with open(pipeline_fname) as f:
    s = f.read()
with open(pipeline_fname, 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train2017)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val2017)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    s = re.sub('fine_tune_checkpoint_type: ".*?"', 'fine_tune_checkpoint_type: "{}"'.format(checkpoint_type), s)
    f.write(s)

In [44]:
# !type {pipeline_fname}

In [10]:
import os
model_dir = 'training/'
# Optionally remove content in output model directory to fresh start.
import shutil
shutil.rmtree(model_dir)
os.makedirs(model_dir, exist_ok=True)

## Train the model

In [11]:
!python models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_fname} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={num_steps} \
    --num_eval_steps={num_eval_steps}

2020-12-20 05:14:30.183284: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-20 05:14:33.461199: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-20 05:14:33.461688: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2020-12-20 05:14:33.494489: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2020-12-20 05:14:33.494506: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-20 05:14:33.497489: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202

## Exporting a Trained Inference Graph
Once your training job is complete, you need to extract the newly trained inference graph, which will be later used to perform the object detection. This can be done as follows:

In [12]:
import re
import numpy as np
import os

output_directory = './fine_tuned_model'
model_dir = 'training/'

lst = os.listdir(model_dir)
lst = [l for l in lst if 'ckpt-' in l]
steps=np.array([int(re.findall('\d+', l)[0]) for l in lst])
last_model = lst[steps.argmax()]

last_model_path = os.path.join(model_dir, last_model)
print(last_model_path)
# !python models/research/object_detection/export_inference_graph.py \
#     --input_type=image_tensor \
#     --pipeline_config_path={pipeline_fname} \
#     --output_directory={output_directory} \
#     --trained_checkpoint_prefix={last_model_path}

!python models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir={model_dir} \
    --pipeline_config_path={pipeline_fname} \
    --output_directory={output_directory}

training/ckpt-11.data-00000-of-00001


2020-12-20 06:02:14.644969: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-20 06:02:17.445759: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-20 06:02:17.446102: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2020-12-20 06:02:17.481350: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2020-12-20 06:02:17.481369: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-20 06:02:17.484534: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202

## Run inference test
Test with images in repository `tensorflow-object-detection/test` directory.

**To test with your own images, you need to place your images inside the `test` directory in this Colab notebook!** More on this below.

In [13]:
import os
import glob

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = "fine_tuned_model/saved_model/saved_model.pb"

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = "class.pbtxt"

# If you want to test the code with your images, just add images files to the PATH_TO_TEST_IMAGES_DIR.
PATH_TO_TEST_IMAGES_DIR =  "dataset/TDB_M_320/Test"

assert os.path.isfile(PATH_TO_CKPT)
assert os.path.isfile(PATH_TO_LABELS)
TEST_IMAGE_PATHS = glob.glob(os.path.join(PATH_TO_TEST_IMAGES_DIR, "*.*"))
assert len(TEST_IMAGE_PATHS) > 0, 'No image found in `{}`.'.format(PATH_TO_TEST_IMAGES_DIR)
print(TEST_IMAGE_PATHS)

['dataset/TDB_M_320/Test\\00063.jpg', 'dataset/TDB_M_320/Test\\00078.jpg', 'dataset/TDB_M_320/Test\\00079.jpg', 'dataset/TDB_M_320/Test\\00081.jpg', 'dataset/TDB_M_320/Test\\00088.jpg', 'dataset/TDB_M_320/Test\\00098.jpg', 'dataset/TDB_M_320/Test\\00099.jpg', 'dataset/TDB_M_320/Test\\00100.jpg', 'dataset/TDB_M_320/Test\\00104.jpg', 'dataset/TDB_M_320/Test\\00108.jpg', 'dataset/TDB_M_320/Test\\00120.jpg', 'dataset/TDB_M_320/Test\\00135.jpg', 'dataset/TDB_M_320/Test\\00136.jpg', 'dataset/TDB_M_320/Test\\00147.jpg', 'dataset/TDB_M_320/Test\\00168.jpg', 'dataset/TDB_M_320/Test\\00189.jpg', 'dataset/TDB_M_320/Test\\00222.jpg', 'dataset/TDB_M_320/Test\\00236.jpg', 'dataset/TDB_M_320/Test\\00244.jpg', 'dataset/TDB_M_320/Test\\00260.jpg', 'dataset/TDB_M_320/Test\\00264.jpg', 'dataset/TDB_M_320/Test\\00289.jpg', 'dataset/TDB_M_320/Test\\00306.jpg', 'dataset/TDB_M_320/Test\\00314.jpg', 'dataset/TDB_M_320/Test\\00347.jpg', 'dataset/TDB_M_320/Test\\00363.jpg', 'dataset/TDB_M_320/Test\\00364.jpg', 

In [14]:
import io
import os
import scipy.misc
import numpy as np
import six
import time
import glob
from IPython.display import display

from six import BytesIO

import matplotlib
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

%matplotlib inline

In [15]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path (this can be local or on colossus)

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(BytesIO(img_data))
    (im_width, im_height) = image.size
    return np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [16]:
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [17]:
tf.keras.backend.clear_session()
model = tf.saved_model.load(f'fine_tuned_model/saved_model/')

In [18]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # Run inference
    model_fn = model.signatures["serving_default"]
    output_dict = model_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    print(output_dict["num_detections"])
    tf.print(output_dict["num_detections"])
#     print(tf.dtypes.cast(output_dict["num_detections"], tf.int32))
#     num_detections = int(output_dict.pop("num_detections"))
    num_detections = int(output_dict.pop("num_detections"))
    output_dict = {
        key: value[0, :num_detections].numpy() for key, value in output_dict.items()
    }
    output_dict["num_detections"] = num_detections

    # detection_classes should be ints.
    output_dict["detection_classes"] = output_dict["detection_classes"].astype(np.int64)

    # Handle models with masks:
    if "detection_masks" in output_dict:
        # Reframe the the bbox mask to the image size.
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            output_dict["detection_masks"],
            output_dict["detection_boxes"],
            image.shape[0],
            image.shape[1],
        )
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5, tf.uint8)
        output_dict["detection_masks_reframed"] = detection_masks_reframed.numpy()

    return output_dict


In [19]:
tf.__version__

'2.4.0'

In [ ]:
for image_path in glob.glob("dataset/TDB_M_splitted/Test/*.jpg"):
    image_np = load_image_into_numpy_array(image_path)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
        image_np,
        output_dict["detection_boxes"],
        output_dict["detection_classes"],
        output_dict["detection_scores"],
        category_index,
        instance_masks=output_dict.get("detection_masks_reframed", None),
        use_normalized_coordinates=True,
        line_thickness=8,
    )
    display(Image.fromarray(image_np))


In [34]:
## Convert Saved Model to Tensorflow JS model (model.json)
! tensorflowjs_converter \
    --input_format=tf_saved_model \
    --output_node_names='detection_boxes,detection_scores,detection_classes,num_detections' \
    --saved_model_tags=serve \
    fine_tuned_model/saved_model \
    fine_tuned_model/tfjs

Writing weight file fine_tuned_model/tfjs\model.json...


2020-12-20 08:44:40.010499: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-20 08:44:47.205894: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2020-12-20 08:44:47.206264: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library nvcuda.dll
2020-12-20 08:44:47.234280: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce RTX 2070 SUPER computeCapability: 7.5
coreClock: 1.785GHz coreCount: 40 deviceMemorySize: 8.00GiB deviceMemoryBandwidth: 417.29GiB/s
2020-12-20 08:44:47.234297: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cudart64_110.dll
2020-12-20 08:44:47.237402: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library cublas64_11.dll
202